In [43]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from pprint import pprint

In [2]:
# Source: https://towardsdatascience.com/how-to-develop-a-credit-risk-model-and-scorecard-91335fc01f03
# https://github.com/finlytics-hub/credit_risk_model
def run_query(query):

    # endpoint for making the request
    # https://api.thegraph.com/subgraphs/name/aave/protocol
    request = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-multy-raw'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [9]:
users_query = """
{
  users(where: {
    id_not_in: ["0x0000000000000000000000000000000000000001",
                "0x000000000000000000000000000000000000dead"]
  }){
    id
    depositHistory(orderBy: timestamp, orderDirection: desc,) {
      id
      amount
      timestamp
      reserve {
        symbol
        name
      }
    }
    borrowHistory(orderBy: timestamp, orderDirection: desc) {
      id
      amount
      timestamp
      reserve {
        symbol
        name
      }
      borrowRate
      borrowRateMode 
      accruedBorrowInterest
    }
  }
}
"""

users_result = run_query(users_query)
#pprint(user_result)

In [10]:
users_result['data']['users']

[{'borrowHistory': [{'accruedBorrowInterest': '0',
    'amount': '150000000000',
    'borrowRate': '100267633691513904610089272',
    'borrowRateMode': 'Stable',
    'id': '0x41b9c8c9ac89432496e35e61b2c97ac947617ad10c778da6fa20d7d9d48d84b7:2',
    'reserve': {'name': 'USD//C', 'symbol': 'USDC'},
    'timestamp': 1603889247}],
  'depositHistory': [{'amount': '15283385447182176619637',
    'id': '0xa2ddce2456750d2d1a4203ef1c24cc257f72af9178b2135fd3f33db617926939:1',
    'reserve': {'name': 'Kyber Network Crystal', 'symbol': 'KNC'},
    'timestamp': 1603890521},
   {'amount': '56761676953101193690201',
    'id': '0xa0ea2be639bf40c4e065be40f3cdff7e2d88a505335e555254c557285f113660:1',
    'reserve': {'name': 'Basic Attention Token', 'symbol': 'BAT'},
    'timestamp': 1603890421},
   {'amount': '1000000000000000000000000',
    'id': '0xb85aa73048f00ca8f6056c514c3c6f639c4f304b6656b70c97dc83dfa2d1e55b:1',
    'reserve': {'name': 'Republic Token', 'symbol': 'REN'},
    'timestamp': 1603889078},

In [29]:
def extract_deposit_history(user_id, deposit=None):
    """
    Extract deposit history from a given user
    """
    
    deposit_dict = dict(
        user_id = user_id,
        deposit_reserve_name = '',
        deposit_reserve_symbol = '',
        deposit_amount = 0,
        deposit_timestamp_unix = None,
        deposit_timestamp = None
    )
    
    if deposit:
        deposit_dict['deposit_reserve_name'] = deposit['reserve']['name']
        deposit_dict['deposit_reserve_symbol'] = deposit['reserve']['symbol']
        deposit_dict['deposit_amount'] = int(deposit['amount'])
        deposit_dict['deposit_timestamp_unix'] = deposit['timestamp']
        deposit_dict['deposit_timestamp'] = pd.to_datetime(deposit['timestamp'], 
                                                           unit='s', errors='ignore')
        
    return deposit_dict

def extract_borrow_history(user_id, borrow=None):
    
    borrow_dict = dict(
        user_id = user_id,
        borrow_reserve_name = '',
        borrow_reserve_symbol = '',
        borrow_amount = 0,
        borrow_timestamp_unix = None,
        borrow_timestamp = None,
        borrow_rate = 0,
        borrow_rate_mode = '',
        borrow_accrued_interest = 0
    )
    
    if borrow:
        borrow_dict['borrow_reserve_name'] = borrow['reserve']['name']
        borrow_dict['borrow_reserve_symbol'] = borrow['reserve']['symbol']
        borrow_dict['borrow_amount'] = int(borrow['amount'])
        borrow_dict['borrow_timestamp_unix'] = borrow['timestamp']
        borrow_dict['borrow_timestamp'] = pd.to_datetime(borrow['timestamp'],
                                                         unit='s', errors='ignore')
        borrow_dict['borrow_rate'] = int(borrow['borrowRate'])
        borrow_dict['borrow_rate_mode'] = borrow['borrowRateMode']
        borrow_dict['borrow_accrued_interest'] = int(borrow['accruedBorrowInterest'])
    
    return borrow_dict



In [30]:
deposits, borrows = [], []
for user in users_result['data']['users']:
    #print(user, '\n')
    
    user_id = user['id']
    
    # extract depositHistory
    if user['depositHistory']:
        for deposit in user['depositHistory']:
            deposit_dict = extract_deposit_history(user_id, deposit)
            deposits.append(deposit_dict)
    else:
        deposit_dict = extract_deposit_history(user_id)
        deposits.append(deposit_dict)
    
    # extract borrowHistory
    if user['borrowHistory']:
        for borrow in user['borrowHistory']:
            borrow_dict = extract_borrow_history(user_id, borrow)
            borrows.append(borrow_dict)
    else:
        borrow_dict = extract_borrow_history(user_id)
        borrows.append(borrow_dict)
   
        
            

In [39]:
deposits_df = pd.DataFrame(deposits)
deposits_df.head()

,user_id,deposit_reserve_name,deposit_reserve_symbol,deposit_amount,deposit_timestamp_unix,deposit_timestamp
0,0x0000000000000d9054f605ca65a2647c2b521422,Kyber Network Crystal,KNC,15283385447182176619637,1.603891e+09,2020-10-28 13:08:41
1,0x0000000000000d9054f605ca65a2647c2b521422,Basic Attention Token,BAT,56761676953101193690201,1.603890e+09,2020-10-28 13:07:01
2,0x0000000000000d9054f605ca65a2647c2b521422,Republic Token,REN,1000000000000000000000000,1.603889e+09,2020-10-28 12:44:38
3,0x0000000000000d9054f605ca65a2647c2b521422,Republic Token,REN,140000000000000000000000,1.603889e+09,2020-10-28 12:42:46
4,0x0000000000000eb4ec62758aae93400b3e5f7f18,,,0,NaN,NaT


In [40]:
borrows_df = pd.DataFrame(borrows)
borrows_df.head()

,user_id,borrow_reserve_name,borrow_reserve_symbol,borrow_amount,borrow_timestamp_unix,borrow_timestamp,borrow_rate,borrow_rate_mode,borrow_accrued_interest
0,0x0000000000000d9054f605ca65a2647c2b521422,USD//C,USDC,150000000000,1.603889e+09,2020-10-28 12:47:27,100267633691513904610089272,Stable,0
1,0x0000000000000eb4ec62758aae93400b3e5f7f18,,,0,NaN,NaT,0,,0
2,0x00000000000017c75025d397b91d284bbe8fc7f2,,,0,NaN,NaT,0,,0
3,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,,,0,NaN,NaT,0,,0
4,0x0000000000009480cded7b47d438e73edf0f67e5,,,0,NaN,NaT,0,,0


In [51]:
no_deposits = set(deposits_df[deposits_df.deposit_amount == 0].user_id)
no_borrows = set(borrows_df[borrows_df.borrow_amount == 0].user_id)

print(len(no_deposits))
print(len(no_borrows))

len(no_deposits.intersection(no_borrows))

19
75


19